In [1]:
import sys

sys.path.append("../../src")

In [2]:
import time

import joblib
import pandas as pd
from fitpredictgo import FitPredictGo, Results
from imblearn.over_sampling import SMOTENC

## Загрузка данных

In [3]:
X_train = pd.read_csv("../../data/baseline/X_train.csv")
X_test = pd.read_csv("../../data/baseline/X_test.csv")
y_train = pd.read_csv("../../data/baseline/y_train.csv")
y_test = pd.read_csv("../../data/baseline/y_test.csv")

## Описание исследования

В данном исследовании (эксперименте) предлагается применить технику оверсемплинга для лучшей модели, построенной с применением алгоритма градиентного бустинга.

## Подготовка данных 

Определим признаки для обучения и целевой признак

In [4]:
original_cat_features = []

original_num_features = [
    "age",
    "systolic_bp",
    "diastolic_bp",
    "glucose_level",
    "body_temperature",
    "heart_rate",
]

new_cat_features = ["age_group", "pressure_group"]

new_num_features = []

target = "risk_level"

cat_features = original_cat_features + new_cat_features
num_features = original_num_features + new_num_features

Установим дополнительную константу для указания количества блоков для кросс-валидации.

In [5]:
cv_folds = 5

In [6]:
scoring = "f1_weighted"

In [7]:
RANDOM_STATE = 1206

Определим перечень моделей для обучения. В словарь в будущем внесем результаты обучения моделей.

In [8]:
names = {
    "CatBoostClassifier()": None,
}

In [9]:
smotenc = SMOTENC(categorical_features=[0, 1], random_state=1206)

In [10]:
preprocessors = {
    "CatBoostClassifier()": [smotenc],
}

Определим инициализационные модели для кросс-валидации

In [11]:
exp_model = joblib.load("../../models/exp3_model_756.joblib")

In [12]:
train_model = exp_model.named_steps["catboostclassifier"]

In [13]:
models = {
    "CatBoostClassifier()": train_model,
}

Выполним обучение. Для каждой модели отображается пайп-лайн для контроля.

In [14]:
%%time

res = Results()
for name in names.keys():
    print(f"{name}, CV")
    model = FitPredictGo(name, scoring)
    start = time.time()
    result = model.fit_cv(
        preprocessors[name],
        models[name],
        X_train,
        y_train["risk_level"],
        cv_folds,
        True,
    )
    time_cv = round(time.time() - start, 3)
    res.update(result)
    names[name] = model
    print(f"--> Done. CV Time = {time_cv} секунд," f"\n\n")
print("All Done")

sorted_results = res.df.sort_values(
    by="Среднее значение метрики при CV", ascending=False
)
sorted_results

CatBoostClassifier(), CV


Pipeline(steps=[('smotenc',
                 SMOTENC(categorical_features=[0, 1], random_state=1206)),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x000002C373967530>)])

0:	learn: 1.0401050	total: 239ms	remaining: 11.7s
25:	learn: 0.4985860	total: 1.19s	remaining: 1.1s
49:	learn: 0.3684153	total: 1.97s	remaining: 0us
0:	learn: 1.0366735	total: 29.9ms	remaining: 1.47s
25:	learn: 0.5393093	total: 859ms	remaining: 793ms
49:	learn: 0.4012407	total: 1.66s	remaining: 0us
0:	learn: 1.0267159	total: 30.6ms	remaining: 1.5s
25:	learn: 0.4983916	total: 878ms	remaining: 810ms
49:	learn: 0.3925773	total: 1.75s	remaining: 0us
0:	learn: 1.0340698	total: 28.1ms	remaining: 1.38s
25:	learn: 0.4892784	total: 855ms	remaining: 789ms
49:	learn: 0.3785372	total: 1.66s	remaining: 0us
0:	learn: 1.0377380	total: 30.7ms	remaining: 1.51s
25:	learn: 0.4858567	total: 857ms	remaining: 791ms
49:	learn: 0.3753318	total: 1.62s	remaining: 0us
0:	learn: 1.0361102	total: 33.7ms	remaining: 1.65s
25:	learn: 0.5058743	total: 862ms	remaining: 796ms
49:	learn: 0.3953036	total: 1.66s	remaining: 0us
--> Done. CV Time = 10.792 секунд,


All Done
CPU times: total: 19.5 s
Wall time: 10.8 s


,Наименование модели,"Время обучения, сек.","Время предсказания, сек.",Среднее значение метрики при CV,СКО метрики при CV
0,CatBoostClassifier(),1.803,0.004,0.742,0.048


In [15]:
best_model_1 = names["CatBoostClassifier()"].get_best_estimator("cv")

In [16]:
print("Лучшая модель и её параметры:\n\n", best_model_1.steps[1][1].get_params())

Лучшая модель и её параметры:

 {'random_seed': 1206, 'verbose': 25, 'n_estimators': 50, 'cat_features': ['age_group', 'pressure_group'], 'learning_rate': 0.1, 'l2_leaf_reg': 2, 'depth': 7, 'auto_class_weights': 'SqrtBalanced'}


## Сохранение модели и данных для дальнейшей работы

Эксперимент не продемонстрировал хороших результатов. Обученная модель не подлежит сохранению.